<a href="https://colab.research.google.com/github/sandilyabaruah3023/Deep-Learning-Assignment-IIT/blob/main/DogsCats_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [ ]:
# Download the dataset (this is a smaller version for quick testing)
!wget https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

# Unzip the dataset
!unzip -q kagglecatsanddogs_5340.zip -d data

# Clean up corrupted images (there are a few in this dataset)
!find data/PetImages/ -type f -print0 | xargs -0 stat -f "%z %N" | grep ^0 | cut -d " " -f 2 | xargs rm

--2025-06-26 13:47:14--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 184.29.22.44, 2600:1406:bc00:e8a::317f, 2600:1406:bc00:e85::317f
Connecting to download.microsoft.com (download.microsoft.com)|184.29.22.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘kagglecatsanddogs_5340.zip’

kagglecatsanddogs_5 100%[===================>] 786.67M   102MB/s    in 7.0s    

2025-06-26 13:47:21 (113 MB/s) - ‘kagglecatsanddogs_5340.zip’ saved [824887076/824887076]

stat: cannot read file system information for '%z %N': No such file or directory
stat: cannot read file system information for '%z %N': No such file or directory
stat: cannot read file system information for '%z %N': No such file or directory
stat: cannot read file system information for '%z %N': No such file or directory
stat: can

In [ ]:
# Create train and validation directories
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

os.makedirs(os.path.join(train_dir, 'Cat'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'Dog'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'Cat'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'Dog'), exist_ok=True)

# Move images to train and validation directories (simplified for demonstration)
# In a real scenario, you would want to split the data more carefully
def move_images(source_dir, train_split=0.8):
    for class_name in ['Cat', 'Dog']:
        class_path = os.path.join(source_dir, class_name)
        images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
        np.random.shuffle(images) # Shuffle images
        train_count = int(len(images) * train_split)

        for i, image_name in enumerate(images):
            src_path = os.path.join(class_path, image_name)
            if i < train_count:
                dest_path = os.path.join(train_dir, class_name, image_name)
            else:
                dest_path = os.path.join(val_dir, class_name, image_name)
            try:
              os.rename(src_path, dest_path)
            except FileNotFoundError:
              print(f"Warning: Could not move {src_path}. It might have been corrupted or removed.")
            except Exception as e:
              print(f"An error occurred while moving {src_path}: {e}")


move_images(data_dir)

print("Data split into train and validation sets.")

Data split into train and validation sets.


In [ ]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'data/PetImages'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                  data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

print(f"Class names: {class_names}")
print(f"Train dataset size: {dataset_sizes['train']}")
print(f"Validation dataset size: {dataset_sizes['val']}")

Class names: ['Cat', 'Dog']
Train dataset size: 20000
Validation dataset size: 4998


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Load a pre-trained ResNet101 model
model_ft = models.resnet101(weights='IMAGENET1K_V1')
num_ftrs = model_ft.fc.in_features

# Modify the final fully connected layer to match the number of classes
# The number of output classes is 2 (Cat and Dog)
model_ft.fc = nn.Linear(num_ftrs, 2)

# Move the model to the appropriate device (GPU if available)
model_ft = model_ft.to(device)

# Define the loss function (Cross-Entropy Loss is common for classification)
criterion = nn.CrossEntropyLoss()

# Define the optimizer (Stochastic Gradient Descent with momentum is a good choice)
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
# This helps in fine-tuning the model by reducing the learning rate over time
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

print("Model, criterion, optimizer, and scheduler defined.")

Model, criterion, optimizer, and scheduler defined.


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model if it's the best accuracy
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:.4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# Start training the model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


train Loss: 0.1058 Acc: 0.9558


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


val Loss: 0.0178 Acc: 0.9942

Epoch 1/9
----------


...

val Loss: 0.0082 Acc: 0.9970

Epoch 9/9
----------


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


train Loss: 0.0335 Acc: 0.9860


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


val Loss: 0.0081 Acc: 0.9966

Training complete in 62m 5s
Best val Acc: 0.9974


In [ ]:
# Evaluate the model
def evaluate_model(model, dataloader, dataset_size, class_names):
    model.eval()  # Set model to evaluate mode
    running_corrects = 0
    all_preds = []
    all_labels = []

    # Iterate over data.
    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        with torch.no_grad():
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        running_corrects += torch.sum(preds == labels.data)

    epoch_acc = running_corrects.double() / dataset_size

    print(f'Evaluation Accuracy: {epoch_acc:.4f}')
    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))
    print("Confusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

evaluate_model(model_ft, dataloaders['val'], dataset_sizes['val'], class_names)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Evaluation Accuracy: 0.9974
Classification Report:
              precision    recall  f1-score   support

         Cat       1.00      1.00      1.00      2499
         Dog       1.00      1.00      1.00      2499

    accuracy                           1.00      4998
   macro avg       1.00      1.00      1.00      4998
weighted avg       1.00      1.00      1.00      4998

Confusion Matrix:
[[2494    5]
 [   8 2491]]


In [ ]:
from PIL import Image

def is_valid_image(image_path):
    try:
        Image.open(image_path).verify()
        return True
    except Exception:
        return False

# Identify and remove corrupted images
corrupted_images = []
for class_name in ['Cat', 'Dog']:
    class_path = os.path.join(data_dir, 'train', class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        if not is_valid_image(img_path):
            corrupted_images.append(img_path)
            os.remove(img_path)
            print(f"Removed corrupted image: {img_path}")

    class_path = os.path.join(data_dir, 'val', class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        if not is_valid_image(img_path):
            corrupted_images.append(img_path)
            os.remove(img_path)
            print(f"Removed corrupted image: {img_path}")

print(f"Identified and removed {len(corrupted_images)} corrupted images.")

Removed corrupted image: data/PetImages/val/Cat/Thumbs.db
Removed corrupted image: data/PetImages/val/Cat/666.jpg


/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Removed corrupted image: data/PetImages/val/Dog/Thumbs.db
Removed corrupted image: data/PetImages/val/Dog/11702.jpg
Identified and removed 4 corrupted images.


In [ ]:
# Save the trained model
model_save_path = 'cats_and_dogs_resnet101.pth'
torch.save(model_ft.state_dict(), model_save_path)

print(f"Trained model saved to {model_save_path}")

Trained model saved to cats_and_dogs_resnet101.pth


In [ ]:
from PIL import Image

def predict_image(model, image_path, transform, class_names, device):
    model.eval()  # Set model to evaluate mode

    try:
        image = Image.open(image_path).convert('RGB')
    except FileNotFoundError:
        print(f"Error: Image file not found at {image_path}")
        return None
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        return None

    # Apply the same transformations as used for validation
    image = transform(image)
    image = image.unsqueeze(0)  # Add a batch dimension
    image = image.to(device)

    with torch.no_grad():
        outputs = model(image)
        _, preds = torch.max(outputs, 1)

    predicted_class = class_names[preds.item()]
    return predicted_class

# Example usage: Replace 'path/to/your/new_image.jpg' with the actual path to a new image
# You can upload an image to your Colab environment and provide the path here.
new_image_path = '/content/drive/MyDrive/cats and dog folder/cat and dog images/photo-1543852786-1cf6624b9987.jpeg' # Replace with your image path

# Use the validation transform for prediction
validation_transform = data_transforms['val']

predicted_class = predict_image(model_ft, new_image_path, validation_transform, class_names, device)

if predicted_class:
    print(f"The image is predicted to be a: {predicted_class}")

The image is predicted to be a: Cat


In [ ]:
from google.colab import drive
drive.mount('/content/drive')